In [ ]:
import os
%pwd

In [ ]:
os.chdir('../')
%pwd

# 1

In [3]:
from dataclasses import dataclass
from pathlib import Path

                               
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    aws_access_id: str
    aws_secrete_key: str
    local_data_file: Path
    unzip_dir: Path
    bucket_name: str
    data_filename: str

# 2

In [4]:
from src.imageSegmentation.constants import *
from src.imageSegmentation.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])           


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            aws_access_id = config.aws_access_id,
            aws_secrete_key = config.aws_secrete_key,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir,
            bucket_name = config.bucket_name,
            data_filename = config.data_filename 
        )
        
        return data_ingestion_config

In [5]:
import os
import urllib.request as request
import zipfile
from src.imageSegmentation import logger
from src.imageSegmentation.utils.common import get_size
import boto3

In [6]:

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                s3_client = boto3.client('s3',
                                            aws_access_key_id = self.config.aws_access_id,
                                            aws_secret_access_key = self.config.aws_secrete_key)
                s3_client.download_file(Bucket= self.config.bucket_name, Key= self.config.data_filename, Filename = self.config.local_data_file)
            except Exception as e:
                raise e
            logger.info("download! with following info")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-31 13:16:13,021: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-31 13:16:13,034: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-31 13:16:13,034: INFO: common: created directory at: artifacts]
[2023-07-31 13:16:13,039: INFO: common: created directory at: artifacts/data_ingestion]
[2023-07-31 13:16:16,675: INFO: 417448044: download! with following info]
